<a href="https://colab.research.google.com/github/Swayamprakashpatel/DE_Interact.github.io/blob/main/Model_DE_Predictor_Multiple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DRUG-EXCIPIENT INTERACTION PREDICTION TOOL**

Dr. Swayamprakash Patel (Email: swayamprakashpatel.ph@charusat.ac.in)


**Model Details:**

Training Accuracy is 0.9898828268051147

Validation Accuracy is 0.9527363181114197

Training loss is 0.020730653777718544

Validation loss is 0.18604408204555511


In [ ]:
#@title Add PubChem Compound ID (CID) for both Drug and Excipient and Run Both Cells (Shift + Enter)
#%%capture
!pip install pubchempy
import tensorflow as tf
import pandas as pd 
import numpy as np 
import io
from tensorflow import keras
import pubchempy as pcp
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model
import time as tm

#Download Model.hdf5 from google drive
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1MJgrc3AzCtlP9-bEUmKFz-PucWes7V_m',
                                    dest_path='/content/sample_data/Model.hdf5',
                                    unzip=False)
#Load Model
model = load_model('/content/sample_data/Model.hdf5', compile= True)

#Compile Fingerprints (1762 Inputs)
Drug_CID =  1983#@param {type:"number"}
Excipient_CID =  1183#@param {type:"number"}

list(pcp.Compound.from_cid(Drug_CID).cactvs_fingerprint)
CID_D = pcp.Compound.from_cid(Drug_CID)
CID_E = pcp.Compound.from_cid(Excipient_CID)

FPD = CID_D.cactvs_fingerprint
FPE = CID_E.cactvs_fingerprint 

List1 = list(FPD)
List2 = list(FPE)
List = List1 + List2
 
t = pd.DataFrame(np.array(List).reshape(-1,len(List)))

dataset1 = t.values
#t.dtype = int
X_Predict = (dataset1[:,0:1762].astype(int))
#print(X_Predict)
#print(X_Predict.shape)
#print(X_Predict.dtype)

#t1 = model.predict(X_Predict)
#print(t1)

t2 = np.round(model.predict(X_Predict))
#print(t2)
 
#Y_prediction = enc.inverse_transform(t2)
#print(Y_prediction, Y_prediction.shape)
Y1 = np.array([['Possibally Compatible'], ['Possibally Incompatible']])
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(Y1)
OneHotEncoder(handle_unknown='ignore')
enc.categories_
Y = enc.transform(Y1).toarray()
Y_prediction = enc.inverse_transform(t2)

#print(Y_prediction)

In [ ]:
#@title Show Result
#print(t2)
from termcolor import colored
if Y_prediction >= 'Possibally Incompatible':
  color = 'red'
else:
  color = 'blue'
print(colored(Y_prediction,color))

**FOR MULTIPLE PREDICTION**

In [ ]:
#@title UPLOAD EXCEL (Predict.xlsx) for Multiple Prediction
# CID to Fingerprint: First Drug
# CID to Fingerprint: First Drug

#Uploade Predict.xlsx file
from google.colab import files
uploaded = files.upload()

df = pd.read_excel('/content/Predict.xlsx')
df1 = df.iloc[:, 0:1]
cidvals = df1['Drug_CID']
componentFingerprint = []
for cid in cidvals:
  c = pcp.Compound.from_cid(cid)
  componentFingerprint.append(c.cactvs_fingerprint)
  tm.sleep(0.1)

CD1 = df1.assign(fp=componentFingerprint)
i = 1
for  componentFingerprint in c.cactvs_fingerprint:
    CD1['fp'] = CD1['fp'].astype(str)
    CD1['fp'+str (i)] = CD1['fp'].str[i-1:i]
    i = i+ 1
#print(componentFingerprint)
#print(CD1)

# CID to Fingerprint: Second Drug
df = pd.read_excel('/content/Predict.xlsx')
df2 = df.iloc[:, 1:2]
cidvals = df2['Excipient_CID']
componentFingerprint = []
for cid in cidvals:
  c = pcp.Compound.from_cid(cid)
  componentFingerprint.append(c.cactvs_fingerprint)
  tm.sleep(0.1)

CD2 = df2.assign(fp=componentFingerprint)
i = 1
for  componentFingerprint in c.cactvs_fingerprint:
    CD2['fp'] = CD2['fp'].astype(str)
    CD2['fp'+str (i)] = CD2['fp'].str[i-1:i]
    i = i+ 1
#print(componentFingerprint)
#print(CD2)

CID= pd.DataFrame(pd.concat([CD1,CD2], axis = 1)) # Two CID Joined
CID_Data = CID.drop(['Drug_CID', 'fp','Excipient_CID'], axis=1, inplace = False)

X_Predict = (CID_Data.iloc[:,0:1762].astype(int))
t2 = np.round(model.predict(X_Predict))
Y1 = np.array([['Possiblly Compatible'], ['Possiblly Incompatible']])
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(Y1)
OneHotEncoder(handle_unknown='ignore')
enc.categories_
Y = enc.transform(Y1).toarray()
Y_prediction = enc.inverse_transform(t2)


In [ ]:
#@title Download Results in Excel
result = pd.DataFrame(Y_prediction)
Intact = pd.DataFrame(pd.concat([df1,df2,result], axis = 1))
Intact.to_excel("Interaction_Result.xlsx")
files.download('Interaction_Result.xlsx') 